In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import ClusterCentroids
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline 

In [3]:
treino = pd.read_csv("train.csv")
teste = pd.read_csv("test.csv")

treino = treino.drop(columns=['media_receita','media_despesa','nome', 'sequencial_candidato'])
teste = teste.drop(columns=['media_receita','media_despesa','nome', 'sequencial_candidato'])

numeric_feats = treino.dtypes[treino.dtypes != "object"].index

numeric_feats = numeric_feats.drop('ano')
numeric_feats = numeric_feats.drop('situacao')

treino[numeric_feats] = np.log1p(treino[numeric_feats])

teste[numeric_feats] = np.log1p(teste[numeric_feats])

allData = pd.concat((teste,treino), sort=False)

teste = allData.loc[:,:'ocupacao']
teste = pd.get_dummies(teste)

treino_X = teste.loc[teste['ano'] != 2014]
teste = teste.loc[teste['ano'] == 2014]

treino_Y = allData.loc[allData['ano'] != 2014]
treino_Y = treino_Y.loc[:,'situacao']

treino_X = treino.loc[:,:'ocupacao']
treino_Y = treino.loc[:,'situacao']

treino_X = pd.get_dummies(treino_X)
teste = pd.get_dummies(teste)

cc = ClusterCentroids()
x_cc, y_cc = cc.fit_sample(treino_X, treino_Y)
y_cc = y_cc[:,np.newaxis]

data = np.concatenate((x_cc, y_cc), axis=1)
data = np.random.permutation(data)

x_cc = data[:,0:-1]

y_cc = data[:,-1]

treino_X = np.array(treino_X)
treino_Y = np.array(treino_Y)

In [11]:
def cv_report(model, predictors, outcome, nfolds):
    
    kf = KFold(n_splits=nfolds)
    f1_score = []
    for train, validation in kf.split(predictors):
        train_predictors = (predictors[train,:])
        train_target =  outcome[train]
        print("FIT")
        model.fit(train_predictors, train_target)
        
        y = model.predict(predictors[validation,:])
        report = metrics.classification_report(outcome[validation], y, output_dict=True)
        f1_score.append(report['micro avg']['f1-score'])
        print(report['micro avg']['f1-score'])
    
    print("F1 = {1}".format(np.mean(f1_score)))
    return np.mean(f1_score)

def real_f1(model):
    return metrics.classification_report(treino_Y, model.predict(treino_X), output_dict=True)['micro avg']['f1-score']

def report_full(model,trainX, trainY, n):
    model.fit(trainX, trainY)
    return real_f1(model)

def polynomial_regression(model,deg):
    polyModel = make_pipeline(PolynomialFeatures(deg),model)
    return polyModel

In [15]:
lambdas = np.arange(0.01,2,0.01).tolist()

cv_logRegBalanced = [cv_report(polynomial_regression(LogisticRegression(solver="lbfgs", max_iter=10000, C=lbd, random_state=1),2), x_cc, y_cc, 5)
                for lbd in lambdas]

cv_logRegBalanced = pd.Series(cv_logRegBalanced, index = lambdas)

cv_logRegBalanced.plot(title = "Cross validation - Logistic Regression {0}".format(i))
plt.xlabel("Lambda")
plt.ylabel("F1-score")
plt.show()

print("O melhor alpha é {0}, com F1 = {1}".format(cv_logRegBalanced.idxmax(), cv_logRegBalanced.max()))

predictions = lgModel.predict(teste)

envio = pd.read_csv("sample_submission.csv")

envio["Predicted"] = predictions
envio["Predicted"] = envio["Predicted"].replace([0, 1],['nao_eleito', 'eleito'])
envio.to_csv("Envio.csv")

FIT
0.8613138686131386
FIT
0.8491484184914841
FIT
0.8414634146341463
FIT


KeyboardInterrupt: 